* 색인 & 가중치 => 학습
1. 수집한 문서 목록 가져오기
2. 각 문서에서 색인어 목록 추출 전처리 -> 어절 -> 형태소+명사+바이그램
3. 가중치 계산(TF-IDF) -> TDM -> TWM(with DocumentVectorLength)

* 질의 -> 색인과정의 2 to 3
4. 질의에서 색인어 추출
5. 가중치 계산
6. 유사도 계산(코싸인)
7. 유사도 순 정렬(거리-오름차순, 각도-내림차순)
8. 검색 결과 출력

In [46]:
def getFileList(base = "./", ext = "txt"): # 아무것도 안했다면 base는 현재 경로
    fileList = list()
    for file in listdir(base):
        if file.split(".")[-1] == ext: # .을 기준으로 split한 것이 txt인지 검사
            fileList.append("{0}/{1}".format(base, file))
            
    return fileList

# txt 컨텐츠를 편하게 읽어오기 위한 함수
def getContent(file): 
    with open(file, encoding="UTF-8") as f:
        content = f.read()
    return content

In [47]:
def ngramEojeol(sentence, n=2): # sentence를 받아 어절 단위로 분리해주는 함수
    '''
    입력:     단어1,   단어2,   단어3,  단어4 : 4
    출력(2) : 단어12,  단어23,  단어34 :        3 - n + 1
    출력(3) : 단어123, 단어234         :        2 - n + 1
    '''
    tokens = sentence.split()
    ngram = []
    
    for i in range(len(tokens) - n + 1):
        ngram.append(' '.join(tokens[i:i + n]))    
        
    return ngram

In [48]:
def ngramUmjeol(term, n = 2): # 음절 단위로 구분하는 함수. sentence를 받아 2개(n=2)씩 쪼갠다.

    ngram = []
    
    for i in range(len(term) - n + 1):
        # ngram.append(tokens_ngram[i:i+n]) # 방법1
        # ngram.append(tuple(tokens_ngram[i:i+n])) # 방법2 (튜플로 반환 시 키값을 쓸 수 있음)
        ngram.append(''.join(term[i:i + n])) # 방법3
        
    return ngram

In [52]:
import re
from string import punctuation


def getPatternList():
    patternList = dict()
    pattern = re.compile(r"[%s]{1,}" % re.escape(punctuation)) # punctuation 안의 특수문자가 두번이상 반복되는 모든 문자에 대해 패턴 정의
    patternList["Punctuation"] = pattern
    pattern2 = re.compile(r"([A-Za-z0-9\-\_\.]{3,}@[A-Za-z0-9\-\_]{3,}(.[A-Za-z]{2,})+)") #이메일주소제거패턴
    patternList["Email"] = pattern2
    pattern3 = re.compile(r"([A-Za-z0-9\-\_]{1,}(.[A-Za-z]{2,})+)") # 신문사도메인패턴
    patternList["Domain"] = pattern3
    pattern4 = re.compile(r"\s{2,}") # 공백제거
    patternList["Whitespace"] = pattern4
    pattern5 = re.compile(r"([^ㄱ-ㅎㅏ-ㅣ가-힣0-9]+)") # 한글이 아닌 영어 기호 제거
    patternList["Korean"] = pattern5
    pattern6 = re.compile(r'[A-Za-z-_]{8,}') # 영어 (대소문자) +\-\_ 제거 (8글자 이상)
    patternList["ElimLongEng"] = pattern6
    pattern7 = re.compile(r"를 2019 및 금지|네이버 채널에서|저작권자 주 블로터앤미디어 저작권자를 명기하고 내용을 변경하지 않으며 비상업적으로 이용하는 조건아래 가능합니다|"
                          r"디지털 마케팅의 미래 아이포럼|무단전재|오류를 우회하기 위한 함수 추가|재배포|기자|아이뉴스24|구독해주세요|"
                          r"고수들의 재테크 비법|이건희칼럼|성공을 꿈꾸는 사람들의 경제 뉴스|머니 및 금지|및 금지|"
                          r"네이버 모바일에서|네이버에서 헤럴드경제|채널 구독하기|헤럴드 리얼라이프 헤럴드경제 사이트 바로가기 헤럴드경제 무단 전재 및 금지|"
                          r"전자신문|바로가기|전자신문인터넷|인터넷 및 금지|중앙일보|친구추가|구독 1위|"
                          r"한국경제|재배포|기사제보|보도자료|한경닷컴|네이버에서|자세히보기|자세히 보기|모바일한경|구독신청|"
                          r"머니투데이|돈이 보이는 리얼타임 뉴스|연예뉴스|파이낸셜뉴스|무단 전재 금지") # 높은 가중치를 갖는 단어를 하드코딩 데이터로 제거
    patternList["ElimRecWord"] = pattern7
    
    return patternList

In [53]:
from os import listdir

FileList = getFileList("C:/Users/brsta/ICT_AI_AdvanceClass_NLP/0314_DownloadedNewstxts/")
NewsContent = list() 

for i in range(len(FileList)):
    NewsContent.append(getContent(FileList[i])) # len(NewsContent) = 180


In [112]:
patternList = getPatternList()
for _ in ["Korean", "Whitespace", "Punctuation", "ElimLongEng", "Email", "Domain", "ElimRecWord"]: 
    for i in range(len(FileList)):
        NewsContent[i] = patternList[_].sub(" ", NewsContent[i]) 
print(NewsContent[1])


 지난 3월11일 유플러스는 한양대학교 에이스 랩 과 공동 개발한 자율주행차를 공개했습니다 유플러스는 를 통해 세계 최초 5 기반의 자율주행차가 일반 차량들과 서울 도심 도로를 달린다 라고 표현했습니다 어딘가 익숙한데요 2017년 평창동계올림픽 기간 동안 는 강릉 평창 등지에서 5 협력 자율주행 버스를 선보였습니다 이때 의 자율주행 버스에도 세계 최초 타이틀이 붙었습니다 텔레콤은 지난해 세계 최초로 복수의 자율주행차가 5 기반 협력 자율주행에 성공했다고 발표했고요 12월1일 텔레콤은 5 전파를 송출하면서 경기도 화성의 자율주행 테스트베드 시티 와 시흥 일반도로에서 5 자율주행차 테스트 운행에 나섰다고 를 통해 밝히기도 했습니다 이 때문에 유플러스 간담회 말미 들의 질문이 쏟아졌습니다 세계 최초가 확실하냐는 거였죠 유플러스는 세계 최초가 맞다 고 확언했습니다 이통사들은 불편한 심기를 내비쳤습니다 텔레콤 관계자는 이미 5 로 자율주행을 한 사례가 있는데 유플러스가 도심에서 자율주행을 했다고 해서 세계 최초 5 자율주행이라 표현하는 건 무리가 있다 라고 지적했습니다 관계자는 남의 집 잔치에 굳이 끼어들고 싶지는 않다 라고 언급을 피했지만 우리도 5 자율주행을 했는데 환경과 범위를 한정하고 좁혀서 세계 최초라고 말하면 그걸 아니라고 할 수는 없다 라는 입장을 밝혔습니다 탐나는 수식어 세계 최초 그래서 누가 세계 최초 5 자율주행을 한 거냐고요 기준에 따라 다릅니다 예를 들어 텔레콤이 지난해 세계 최초라고 했던 건 5 자율주행차가 서로의 운행 경로를 공유하면서 협력 운행에 성공한 것은 이번이 세계 처음 이라던 거였는데요 유플러스의 자율주행은 5 상용 기지국과 연결된 상태로 도심도로에서 자율주행한 세계 최초의 사례라는 겁니다 강종오 유플러스 부문 미래기술담당은 5 통신기술을 상용화해 실제 도심에서 자율주행에 이용한 것은 이번이 세계 최초 라며 유플러스는 통신사 중 5 를 가장 빠르고 촘촘하게 준비하고 있다 라고 말했습니다 참고기사 차 2대로 세계 첫 5 자율주행 우리가 

In [130]:
from nltk.tokenize import sent_tokenize, word_tokenize
from konlpy.tag import Kkma
ma = Kkma()

# 단일 뉴스 기사를 토큰화해주는 함수
def Tokenizer(NewsContent):
    dictTerm = list()
    dictPos = list()
    dictNoun = list()
    dictNgram = list()
    
    for sentence in sent_tokenize(NewsContent):
        for token in word_tokenize(sentence):
            if len(token) > th:
                dictTerm.append(token)
                # 아래서부터는 list이기때문에 extend를 사용해야 함. 
                dictPos.extend([morpheme for morpheme in ma.morphs(token) if len(morpheme) > th]) # 형태소 분석결과를 extend
                dictNoun.extend([noun for noun in ma.nouns(token) if len(noun) > th]) # 명사 단위로 잘라 extend
                dictNgram.extend(ngramUmjeol(token)) # 바이그램을 리턴
                
    # 빠른 속도 및 중복 데이터 처리를 위해 set사용 (유일한 단어만 남김)
    dictTerm = list(set(dictTerm))
    dictPos = list(set(dictPos))
    dictNoun = list(set(dictNoun))
    dictNgram = list(set(dictNgram))
        
    return dictTerm, dictPos, dictNoun, dictNgram

In [132]:
TokenizedNewsContent = list()

for i in range(len(FileList)):
    TokenizedNewsContent.append(Tokenizer(NewsContent[i]))

In [154]:
# len(dictTerm), len(dictPos), len(dictNoun), len(dictNgram) of 0th txt file
print(len(TokenizedNewsContent[0][0]), len(TokenizedNewsContent[0][1]), len(TokenizedNewsContent[0][2]), len(TokenizedNewsContent[0][3]))

# dictTerm, dictPos, dictNoun, dictNgram
for i in range(len(FileList)):
    print(str(i) + "th txt : " + str(len(list(set(TokenizedNewsContent[i][0] + TokenizedNewsContent[i][1] + TokenizedNewsContent[i][2] + TokenizedNewsContent[i][3])))))

276 214 181 484
0th txt : 754
1th txt : 1109
2th txt : 567
3th txt : 242
4th txt : 320
5th txt : 317
6th txt : 681
7th txt : 1120
8th txt : 1453
9th txt : 611
10th txt : 552
11th txt : 381
12th txt : 642
13th txt : 593
14th txt : 465
15th txt : 441
16th txt : 531
17th txt : 674
18th txt : 1133
19th txt : 865
20th txt : 598
21th txt : 448
22th txt : 496
23th txt : 813
24th txt : 654
25th txt : 584
26th txt : 440
27th txt : 284
28th txt : 464
29th txt : 668
30th txt : 403
31th txt : 990
32th txt : 1298
33th txt : 507
34th txt : 495
35th txt : 261
36th txt : 690
37th txt : 890
38th txt : 679
39th txt : 653
40th txt : 942
41th txt : 1349
42th txt : 934
43th txt : 436
44th txt : 1074
45th txt : 398
46th txt : 1144
47th txt : 819
48th txt : 1289
49th txt : 435
50th txt : 1228
51th txt : 661
52th txt : 621
53th txt : 743
54th txt : 752
55th txt : 517
56th txt : 1184
57th txt : 832
58th txt : 1577
59th txt : 639
60th txt : 929
61th txt : 916
62th txt : 846
63th txt : 796
64th txt : 1714
65th t

In [159]:
# 0 : dictTerm, 1 : dictPos, 2 : dictNoun, 3 : dictNgram
print(TokenizedNewsContent[0][0])

['실제', '서명했다고', '있다', '전자기장', '넘는', '단체는', '국내', '이엠에프사이언티스트에', '밝혔다', '전했다', '논란이', '과학자들에게', '대략', '분들', '존재하지만', '제품이나', '여기에는', '이메일로', '동아사이언스가', '내용에', '아직까지', '관한', '이어폰의', '보도가', '세계보건기구', '비이온화', '이어폰에', '자기장에', '제출됐던', '실제로', '노출로부터', '확산되고', '4년전', '나오면서', '과학자', '한양대', '과학자들은', '건강', '작성을', '법적', '알려진', '서명했다', '언급하고', '건강과', '1212', '제조사를', '우리', '낮은', '확인한', '확실히', '늘어났다', '한국의', '제출된', '않다', '포함됐다고', '연구자도', '몰랐다', '학술지에', '된다는', '최근', '한림대', '유발할', '말했다', '주변에', '화학과', '연세대', '캘리포니아대', '관련', '안정성에', '검증됐다', '이미', '교수가', '참여', '있지만', '247명이', '관여하지', '있다는', '호소문에', '이엠에프사이언티스트', '전화나', '저명한', '발생', '호소문에는', '결과', '송기원', '생물학', '서명하라', '247명으로', '인체에', '미국', '어리둥절', '과학자들의', '준수하고', '부정하며', '여러', '지나다니고', '위험을', '관련된', '전세계', '2년전', '담고', '않았고', '5월', '서명자인', '연구들을', '문제가', '다른', '이어폰이', '방지가', '포함됐다', '것도', '조엘', '가정사회건강연구소', '데일리메일과', '단체측', '지속적으로', '현재는', '2000개가', '연구들이', '가까이', '이름을', '제출됐으며', '애플', '전자기장의', '했다고', '전자기장과', '아닌', '시의', '이들', '호소문을', '인터뷰에서', '올라간', 

In [162]:
collection = []
# print(type(FileList[0]))

for i in range(len(NewsContent)):
    collection.append((str(FileList[i]), NewsContent[i]))
print(collection[0])

('C:/Users/brsta/ICT_AI_AdvanceClass_NLP/0314_DownloadedNewstxts//IT과학-0000003610.txt', ' 오히려 단체측 무선이어폰 연구는 존재치도 않아 국내 참여 연구자도 어리둥절 이엠에프사이언티스에서 일부 언론 보도를 부정하며 직접 보내온 이메일 내용이다 이엠에프사이언티스트 관계자인 조엘 모스코위츠 미국 버클리 캘리포니아대 가정사회건강연구소 소장은 무선 이어폰의 건강 유해성에 대한 보도를 부정했다 고재원 1212 18일 오전 전세계 과학자들이 애플 에어팟과 같은 무선 이어폰이 암 발생 위험을 키울 수 있다는 호소문을 국제연합 과 세계보건기구 에 제출했다는 일부 국내외 언론의 보도가 나오면서 불안감이 확산되고 있다 하지만 실제 이 단체와 호소문에 이름을 올린 과학자들에게 확인한 결과 호소문은 4년전 제출됐던 것이며 또 특정 제품이나 제조사를 언급하지 않은 것으로 확인됐다 18일 데일리메일과 등 국내외 일부 언론에 따르면 전 세계 과학자 247명이 무선 이어폰의 비이온화 전자기장 이 암을 유발할 위험 우려가 있다며 과 에 호소문을 제출했다고 전했다 이들 매체들은 호소문에 애플 에어팟이 에 관한 법적 기준치를 준수하고 있지만 장시간 노출될 경우 건강에 좋지 않을 수 있다 는 내용이 포함됐다고 보도했다 이들 매체들은 이 호소문에는 전 세계 42개국 과학자 247명이 서명을 했다고 전했다 여기에는 한국의 연세대 한양대 가톨릭대 단국대 중앙대 경북대 한림대 소속 과학자 15명의 이름도 포함됐다 하지만 취재 결과 호소문 작성을 주도한 비영리단체 이엠에프사이언티스트 는 애플 에어팟과 같은 무선 이어폰에 대한 유해성을 주장하지 않은 것으로 확인됐다 동아사이언스가 이엠에프사이언티스트에 직접 이메일로 확인한 결과 일부 언론 보도가 호소문에 대한 부정확한 내용을 담고 있다 며 무선 블루투스의 자기장에 머리가 장시간 노출될 시의 안정성에 대한 연구는 존재하지 않는다 고 밝혔다 이엠에프사이언티스트에 따르면 이 단체는 지난 201

In [ ]:
# WIP(2019-03-29 10:28 PM)

In [82]:
for (docName, docContent) in collection:
    docIdx = len(globalDocument)
    globalDocument.append(docName)
    
    localPosting = dict()
    
    for term in docContent.lower().split():
        if term not in localPosting.keys():
            localPosting[term] = 1
        else:
            localPosting[term] += 1
    
    maxFreq = max(localPosting.values())
    
    for indexTerm, termFreq in localPosting.items():
        if indexTerm not in globalLexicon.keys():
            lexiconIdx = len(globalLexicon)
            postingIdx = len(globalPosting)
            postingData = [lexiconIdx, docIdx, doubleNormalTF(0, termFreq, maxFreq), -1]
            globalPosting.append(postingData)
            globalLexicon[indexTerm] = postingIdx
        else:
            lexiconIdx = list(globalLexicon.keys()).index(indexTerm)
            postingIdx = len(globalPosting)
            beforeIdx = globalLexicon[indexTerm]
            postingData = [lexiconIdx, docIdx, doubleNormalTF(0, termFreq, maxFreq), beforeIdx]
            globalPosting.append(postingData)
            globalLexicon[indexTerm] = postingIdx

In [83]:
globalPosting

[[0, 0, 0.14285714285714285, -1],
 [1, 0, 0.14285714285714285, -1],
 [2, 0, 0.14285714285714285, -1],
 [3, 0, 0.2857142857142857, -1],
 [4, 0, 0.14285714285714285, -1],
 [5, 0, 0.14285714285714285, -1],
 [6, 0, 0.14285714285714285, -1],
 [7, 0, 0.14285714285714285, -1],
 [8, 0, 0.14285714285714285, -1],
 [9, 0, 0.14285714285714285, -1],
 [10, 0, 0.14285714285714285, -1],
 [11, 0, 0.5714285714285714, -1],
 [12, 0, 0.2857142857142857, -1],
 [13, 0, 0.2857142857142857, -1],
 [14, 0, 0.14285714285714285, -1],
 [15, 0, 0.2857142857142857, -1],
 [16, 0, 0.14285714285714285, -1],
 [17, 0, 0.2857142857142857, -1],
 [18, 0, 0.14285714285714285, -1],
 [19, 0, 0.2857142857142857, -1],
 [20, 0, 0.2857142857142857, -1],
 [21, 0, 0.2857142857142857, -1],
 [22, 0, 0.2857142857142857, -1],
 [23, 0, 0.2857142857142857, -1],
 [24, 0, 0.2857142857142857, -1],
 [25, 0, 0.2857142857142857, -1],
 [26, 0, 0.2857142857142857, -1],
 [27, 0, 0.2857142857142857, -1],
 [28, 0, 0.7142857142857143, -1],
 [29, 0, 0.